## Import libraries

In [3]:
import numpy as np
import pandas as pd
import sys
sys.path.append('..')
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from cuml import RandomForestRegressor as cuRF
import xgboost as xgb
import math
import time
import re
from scripts.clustering_utils import read_clusters_from_file, cluster_predict, aggregate_cluster_predictions, dataframe_by_cluster, split_into_clusters, select_past_timesteps

In [4]:
n_clusters = 16

## Read Dataset

In [5]:
df = pd.read_csv("~/datasets/Dataset.csv", decimal=",")
df_index = pd.DataFrame(pd.date_range('2019-01-01', periods=35136, freq='15T'))
df = pd.concat([pd.DataFrame(df_index).rename(columns={0: "Time"}),df],axis=1)
df

,Time,0,1,2,3,4,5,6,7,8,...,41,42,43,44,45,46,47,48,49,50
0,2019-01-01 00:00:00,2.964,2.322959,1.544607,0.778310,1.962012,2.677445,0.237877,0.689194,0.358525,...,0.898895,0.203825,0.221624,0.319531,0.830996,0.924987,0.219128,0.274880,0.990488,0.779475
1,2019-01-01 00:15:00,2.584,2.371797,1.544607,0.778310,1.962012,2.733737,0.192929,0.558967,0.358525,...,0.917793,0.165311,0.179747,0.319531,0.848467,0.944434,0.177722,0.222940,1.011313,0.795863
2,2019-01-01 00:30:00,3.071,2.415961,1.319880,0.665072,1.676555,2.784640,0.382869,1.109272,0.377198,...,0.934883,0.328060,0.356708,0.336174,0.864266,0.962019,0.352691,0.442426,1.030144,0.810682
3,2019-01-01 00:45:00,2.694,2.302538,1.319880,0.665072,1.676555,2.653908,0.442052,1.280743,0.377198,...,0.890992,0.378772,0.411848,0.336174,0.823691,0.916855,0.407209,0.510816,0.981781,0.772623
4,2019-01-01 01:00:00,2.569,2.363063,0.913154,0.460128,1.159919,2.723669,0.192242,0.556976,0.668500,...,0.914413,0.164722,0.179106,0.595793,0.845343,0.940956,0.177089,0.222146,1.007588,0.792932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35131,2020-01-01 22:45:00,1.664,2.244719,1.455982,0.733653,1.849437,2.587266,0.205654,0.595835,0.171793,...,0.868619,0.176214,0.191602,0.153109,0.803007,0.893832,0.189444,0.237645,0.957128,0.753222
35132,2020-01-01 23:00:00,1.659,2.136340,1.201186,0.605264,1.525786,2.462348,0.201219,0.582985,0.067223,...,0.826680,0.172414,0.187470,0.059912,0.764237,0.850676,0.185359,0.232519,0.910916,0.716855
35133,2020-01-01 23:15:00,1.664,2.192805,1.201186,0.605264,1.525786,2.527430,0.228585,0.662271,0.067223,...,0.848530,0.195862,0.212966,0.059912,0.784436,0.873160,0.210568,0.264142,0.934992,0.735802
35134,2020-01-01 23:30:00,1.697,1.446083,0.259545,0.130782,0.329682,1.666757,0.189302,0.548459,0.070958,...,0.559578,0.162203,0.176368,0.063241,0.517310,0.575820,0.174381,0.218749,0.616596,0.485237


In [6]:
names = ["Linear Regression", "XGBoost", "Random Forest"]

In [10]:
## Complete script
sourceFile = open("../../gpu_logs/cluster_preds_porto___.txt", "w")
all_clusters = read_clusters_from_file("clusters_porto_new.txt", n_clusters)
df_houses = df.drop("Time", axis=1)
estimators = [LinearRegression(), xgb.XGBRegressor(tree_method='gpu_hist', seed=0, colsample_bytree=0.7, learning_rate=0.1, max_depth=10, n_estimators=1000), cuRF()]
all_cluster_preds = []
n_c = 2
for c in all_clusters:
    print("Number of Clusters: {}".format(n_c))
    print("Number of Clusters: {}".format(n_c), file=sourceFile)
    cluster_houses = split_into_clusters(c)
    i = 0
    pred_list = []
    clusters_preds, clusters_actuals = [], []
    for clust in cluster_houses:
        print("Cluster {}".format(i))
        i += 1
        df_cluster = select_past_timesteps("../../data/porto_full_selected.csv", clust)
        c_pred, c_test = cluster_predict(df_cluster, estimators=estimators, names=names)
        clusters_preds.append(c_pred)
        clusters_actuals.append(c_test)
    n_c += 1
    agg_pred = aggregate_cluster_predictions(clusters_preds, clusters_actuals, names, sourceFile)
    all_cluster_preds.append(agg_pred)
sourceFile.close()

Number of Clusters: 2
Cluster 0
Elapsed time training and predicting: 0.0624 s
Elapsed time training and predicting: 6.7022 s


/home/francisco.monteiro@av.it.pt/miniconda3/envs/rapids-23.02/lib/python3.8/site-packages/cuml/internals/api_decorators.py:190: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


Elapsed time training and predicting: 1.4429 s
Cluster 1
Elapsed time training and predicting: 0.2300 s
Elapsed time training and predicting: 8.4618 s


/home/francisco.monteiro@av.it.pt/miniconda3/envs/rapids-23.02/lib/python3.8/site-packages/cuml/internals/api_decorators.py:190: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


Elapsed time training and predicting: 4.3335 s
Aggregating predictions...

----------------------------
Linear Regression
----------------------------

MSE: 0.1967
RMSE: 0.4435
WAPE: 29.69
R2: 0.8297

----------------------------
XGBoost
----------------------------

MSE: 0.0938
RMSE: 0.3063
WAPE: 19.66
R2: 0.9188

----------------------------
Random Forest
----------------------------

MSE: 0.1316
RMSE: 0.3628
WAPE: 24.10
R2: 0.8860
Number of Clusters: 3
Cluster 0
Elapsed time training and predicting: 0.1689 s
Elapsed time training and predicting: 8.0681 s


...

/home/francisco.monteiro@av.it.pt/miniconda3/envs/rapids-23.02/lib/python3.8/site-packages/cuml/internals/api_decorators.py:190: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


Elapsed time training and predicting: 0.4029 s
Cluster 14
Elapsed time training and predicting: 0.0100 s
Elapsed time training and predicting: 5.4196 s


/home/francisco.monteiro@av.it.pt/miniconda3/envs/rapids-23.02/lib/python3.8/site-packages/cuml/internals/api_decorators.py:190: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


Elapsed time training and predicting: 0.5996 s
Cluster 15
Elapsed time training and predicting: 0.0035 s
Elapsed time training and predicting: 3.2829 s
Elapsed time training and predicting: 0.3987 s
Aggregating predictions...

----------------------------
Linear Regression
----------------------------

MSE: 0.1737
RMSE: 0.4168
WAPE: 29.23
R2: 0.8432

----------------------------
XGBoost
----------------------------

MSE: 0.0683
RMSE: 0.2614
WAPE: 15.15
R2: 0.9383

----------------------------
Random Forest
----------------------------

MSE: 0.0981
RMSE: 0.3133
WAPE: 19.85
R2: 0.9114


/home/francisco.monteiro@av.it.pt/miniconda3/envs/rapids-23.02/lib/python3.8/site-packages/cuml/internals/api_decorators.py:190: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


In [44]:
df_selected_features = pd.read_csv("../../data/porto_full_selected.csv")
df_selected_features

,Time,Location,lag_1,lag_2,lag_3,lag_4,lag_96,lag_192,lag_288,lag_384,lag_480,lag_576,lag_672,DayOfWeek,Hour,Energy
0,2019-01-08 00:00:00,0,3.952000,3.803000,4.095000,3.926000,2.615000,3.321000,3.066000,2.351000,2.851000,3.054000,2.964000,1,0,4.940000
1,2019-01-08 00:15:00,0,4.940000,3.952000,3.803000,4.095000,2.634000,2.873000,3.152000,2.843000,3.102000,2.677000,2.584000,1,0,4.149000
2,2019-01-08 00:30:00,0,4.149000,4.940000,3.952000,3.803000,3.052000,3.407000,2.633000,2.949000,2.546000,3.522000,3.071000,1,0,4.401000
3,2019-01-08 00:45:00,0,4.401000,4.149000,4.940000,3.952000,2.893000,2.673000,2.590000,2.605000,3.016000,2.729000,2.694000,1,0,4.431000
4,2019-01-08 01:00:00,0,4.431000,4.401000,4.149000,4.940000,2.349000,2.390000,2.330000,3.023000,3.031000,2.368000,2.569000,1,1,3.988000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1757659,2020-01-01 22:45:00,50,1.099100,0.453906,0.481770,0.483173,0.641933,0.778195,0.753221,0.641933,0.778195,0.753221,1.723943,2,22,0.753221
1757660,2020-01-01 23:00:00,50,0.753221,1.099100,0.453906,0.481770,0.752850,0.931837,0.716855,0.752850,0.931837,0.716855,1.482914,2,23,0.716855
1757661,2020-01-01 23:15:00,50,0.716855,0.753221,1.099100,0.453906,0.935428,0.937079,0.735802,0.935428,0.937079,0.735802,1.772034,2,23,0.735802
1757662,2020-01-01 23:30:00,50,0.735802,0.716855,0.753221,1.099100,0.715781,0.803087,0.485237,0.715781,0.803087,0.485237,2.535657,2,23,0.485237


## Best predictions

In [7]:
with open('../../gpu_logs/cluster_preds_porto.txt', 'r') as f:
    l = f.readlines()
    lines = [s for s in l if re.search(r'\w+', s)]

In [8]:
def get_cluster_metrics(lines):
    cursor = 0
    model_names, mse, wape, r2, number_clust = [], [], [], [], []
    for i in lines:
        if i.startswith("Number"):
            number_clust.append(int(i[-3:-1].strip()))
        else:
            if cursor == 0:
                model_names.append(i[:-1])
                cursor += 1
            elif cursor == 1:
                mse.append(i.split(":")[1][:-1])
                cursor += 1
            elif cursor == 2:
                wape.append(i.split(":")[1][:-1])
                cursor += 1
            elif cursor == 3:
                r2.append(i.split(":")[1][:-1])
                cursor = 0
    mse = list(map(float, mse))
    wape = list(map(float, wape))
    wape = list(map(lambda x: x/100, wape))
    r2 = list(map(float, r2))
    return model_names, mse, wape, r2, number_clust

In [9]:
def best_number_of_cluster(lines):
    model_names, mse, wape, r2, number_clust = get_cluster_metrics(lines)
    min_mse, min_index_mse, mod_index_mse = min(mse), int(np.argmin(mse)/3), np.argmin(mse)%3
    min_wape, min_index_wape, mod_index_wape = min(wape), int(np.argmin(wape)/3), np.argmin(wape)%3
    max_r2, max_index_r2, mod_index_r2 = max(r2), int(np.argmax(r2)/3), np.argmax(r2)%3
    return max_r2, number_clust[max_index_r2], model_names[mod_index_r2]

In [10]:
best_score, best_n_clust, best_alg = best_number_of_cluster(lines)
best_score, best_n_clust, best_alg

(0.9433, 11, 'XGBoost')

In [13]:
model_names, mse, wape, r2, number_clust = get_cluster_metrics(lines)
mse[(best_n_clust-2)*3+1], r2[(best_n_clust-2)*3+1], wape[(best_n_clust-2)*3+1]

(0.0619, 0.9433, 0.15130000000000002)